In [1]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# prepare the data once again
import csv
filename = "/Users/andreamonacchi/Desktop/funjokes.csv"
raw_text = open(filename)
# skip header
raw_text.next()
reader = csv.DictReader(raw_text,fieldnames=['id','joke'])

# create a dataset from the lowercase version of all jokes
in_ds = []
for row in reader:
    in_ds.append(row['joke'].lower())

text = "\n".join(in_ds)
rtext = " ".join(in_ds)
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
n_chars = len(text)
n_vocab = len(chars)
dataX = []
dataY = []

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
for i in range(0, n_chars - seq_length, 1):
    seq_in = rtext[i:i + seq_length]
    seq_out = rtext[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  1944100


In [4]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
from keras.layers import recurrent
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
#model.add(LSTM(units=256, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
#model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [9]:
# load the network weights
filename = "weights-improvement-06-2.0524.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."

Seed:
" e said "claudia schiffer's breasts." while the pope was visiting the usa he told the driver of his l "
and and she was a boo and the man said "io the cart in the car and toent the sare and toeet a coonee of toeet and the saare and toent the saare and the saari said "io the cart in the car and toent the sare and toeet a coonee of toeet and